In [1]:
import PbPMethods2 as pm2
import pandas as pd
root = '/Users/muneebalam/Downloads/'
filenames = [root + x for x in ['CAR_NJ_11_6_16.xlsx', 
                                'BOS_NJ_10_20_16.xlsx']]

Load data

In [2]:
dfs = []
for filename in filenames:
    dfs.append(pd.read_excel(filename, sheetname = 0))
df = pd.concat(dfs)
df = df.dropna(subset=['Period'])
df.head()

,ATG?,Active D,Away:,Breakout Lane?,Breakout Result,Breakout Support,Breakout Type?,Date,Entry Type Allowed,F1,...,Forecheck Used,Forechecking Team,Game ID,Home Score State,Home:,Oddman Against?,PBP Time,Period,Time,Unnamed: 20
0,NaN,NaN,N.J,Left,Controlled,3,Reverse,2016-11-06,Controlled,11,...,212,N.J,20176,0,CAR,NaN,00:11:00,1,19:49:00,20:00
1,NaN,NaN,N.J,Left,Turnover,3,Up,2016-11-06,NaN,25,...,212,CAR,20176,0,CAR,NaN,00:58:00,1,19:02:00,20:00
2,NaN,NaN,N.J,Right,Controlled,3,Up,2016-11-06,Uncontrolled,14,...,Rebound,N.J,20176,0,CAR,NaN,03:36:00,1,16:24:00,20:00
3,NaN,NaN,N.J,Right,Failed Exit,2,Up,2016-11-06,NaN,49,...,Turnover,CAR,20176,0,CAR,NaN,03:51:00,1,16:09:00,20:00
4,NaN,NaN,N.J,Left,Failed Exit,2,Over,2016-11-06,NaN,49,...,Turnover,CAR,20176,0,CAR,NaN,04:00:00,1,16:00:00,20:00


Keep only columns I want and switch to lists

In [3]:
gameids = [x for x in df['Game ID']]
pers = [x for x in df.Period]
times = [x for x in df.Time]
fteams = [x for x in df['Forechecking Team']]
fused = [x for x in df['Forecheck Used']]
foreagg = [x for x in df['Forecheck Aggression']]
breakouts = [x for x in df['Breakout Type?']]
breakoutsupport = [x for x in df['Breakout Support']]

In [4]:
forechecks = []
for i in range(len(df)):
    forechecks.append({'Game ID': int(gameids[i]), 
                       'Period': int(pers[i]), 
                       'Time': times[i], 
                       'Forechecking Team': fteams[i], 
                       'Forecheck Used': fused[i], 
                       'Forecheck Aggression': int(foreagg[i]), 
                       'Breakout Type?': breakouts[i], 
                       'Breakout Support': int(breakoutsupport[i])})
    forechecks[-1]['Min'] = int(str(times[i]).split(':')[0])
    forechecks[-1]['Sec'] = int(str(times[i]).split(':')[1])
    forechecks[-1]['Time2'] = pers[i] * 1200 - forechecks[-1]['Min'] * 60 - forechecks[-1]['Sec']
    forechecks[-1]['Team'] = forechecks[-1]['Forechecking Team']
    forechecks[-1]['Type'] = 'Forecheck'

Create dataframe of all shots

In [5]:
pbp = []
for game in df['Game ID'].value_counts().index:
    game = int(game)
    for line in pm2.read_game_pbp(2016, game, types=['GOAL', 'SHOT', 'MISS', 'BLOCK', 'FAC'], strengths=['all']):
        pbp.append({'Time2': pm2.convert_time(pm2.get_event_time(line), pm2.get_event_period(line)),
                    'Game ID': game,
                    'Team': pm2.get_acting_team(line),
                    'Type': pm2.get_event_type(line)})
print(len(pbp))

364


Combine into one dataframe and sort by time

In [6]:
events = forechecks + pbp
events = sorted(events, key=lambda x: x['Time2'])

For each forecheck, count how many shots came by that same team after, before the next forecheck, next faceoff, or next opposing team shot

In [7]:
for i in range(len(events)):
    cf = 0
    for j in range(i + 1, len(events)):
        if events[j]['Type'] in {'FAC', 'Forecheck'}:
            break
        elif events[j]['Team'] != events[i]['Team']:
            break
        elif events[j]['Game ID'] != events[i]['Game ID']:
            break
        else:
            cf += 1
    events[i]['CF'] = cf
finaldf = pd.DataFrame([x for x in events if x['Type'] == 'Forecheck'])
finaldf.head()

,Breakout Support,Breakout Type?,CF,Forecheck Aggression,Forecheck Used,Forechecking Team,Game ID,Min,Period,Sec,Team,Time,Time2,Type
0,2,Goalie,0,3,32,BOS,20051,19,1,53,BOS,19:53:00,7,Forecheck
1,3,Reverse,0,3,212,N.J,20176,19,1,49,N.J,19:49:00,11,Forecheck
2,2,Up,0,2,Faceoff,BOS,20051,19,1,46,BOS,19:46:00,14,Forecheck
3,2,Reverse,0,2,Turnover,BOS,20051,19,1,37,BOS,19:37:00,23,Forecheck
4,3,Up,0,3,32,N.J,20051,19,1,30,N.J,19:30:00,30,Forecheck


In [8]:
forechecksummary = finaldf[['Forecheck Used', 'Forecheck Aggression', 'CF']].groupby(
    ['Forecheck Used', 'Forecheck Aggression']).agg(['mean', 'count'])
forechecksummarybyteam = finaldf[['Team', 'Forecheck Used', 'Forecheck Aggression', 'CF']].groupby(
    ['Team', 'Forecheck Used', 'Forecheck Aggression']).agg(['mean', 'count'])
forechecksummary

CF      
                                         mean count
Forecheck Used Forecheck Aggression                
32             3                     0.250000     8
113            2                     0.000000     1
122            1                     0.000000    12
               2                     0.000000    14
               3                     0.000000     2
212            1                     0.000000     1
               2                     0.177778    45
               3                     0.166667    18
Faceoff        1                     0.166667     6
               2                     0.375000     8
               3                     1.000000     1
Rebound        1                     0.000000     4
               2                     0.375000    16
               3                     0.266667    15
               4                     2.000000     1
Turnover       1                     0.500000     2
               2                     0.190476    21
               3                     0.454545    11

In [9]:
forechecksummarybyteam

CF      
                                              mean count
Team Forecheck Used Forecheck Aggression                
BOS  32             3                     0.000000     4
     113            2                     0.000000     1
     122            1                     0.000000     2
                    2                     0.000000     4
     212            2                     0.272727    11
                    3                     0.000000     6
     Faceoff        1                     0.000000     3
                    2                     0.333333     3
     Rebound        1                     0.000000     4
                    2                     1.000000     4
                    3                     0.000000     1
     Turnover       2                     0.000000     4
                    3                     0.000000     1
CAR  122            1                     0.000000     6
                    2                     0.000000     5
                    3                     0.000000     1
     212            2                     0.153846    13
                    3                     1.000000     2
     Faceoff        1                     0.000000     2
                    2                     1.000000     1
     Rebound        2                     0.400000     5
                    3                     0.166667     6
                    4                     2.000000     1
     Turnover       1                     0.000000     1
                    2                     0.400000     5
                    3                     0.500000     6
N.J  32             3                     0.500000     4
     122            1                     0.000000     4
                    2                     0.000000     5
                    3                     0.000000     1
     212            1                     0.000000     1
                    2                     0.142857    21
                    3                     0.100000    10
     Faceoff        1                     1.000000     1
                    2                     0.250000     4
                    3                     1.000000     1
     Rebound        2                     0.000000     7
                    3                     0.375000     8
     Turnover       1                     1.000000     1
                    2                     0.166667    12
                    3                     0.500000     4

In [10]:
breakoutsummary = finaldf[['Breakout Type?', 'Breakout Support', 'CF']].groupby(
    ['Breakout Type?', 'Breakout Support']).agg(['mean', 'count'])
breakoutsummary

CF      
                                     mean count
Breakout Type? Breakout Support                
Controlled     2                 0.000000     4
               3                 0.166667     6
Goalie         2                 0.000000     5
               3                 0.200000     5
               4                 0.000000     1
Over           2                 0.000000     9
               3                 0.000000     7
               4                 0.500000     2
               5                 0.000000     2
Reverse        2                 0.500000     4
               3                 0.125000     8
               4                 0.000000     1
Rim            1                 0.000000     1
               2                 0.333333     3
               3                 0.200000    10
               4                 0.000000     1
Up             1                 0.000000     2
               2                 0.043478    23
               3                 0.285714    42
               4                 0.375000     8
               5                 0.000000     1
Wheel          3                 0.000000     1